In [2]:
import stim
import pymatching as pm
import numpy as np
from IPython.display import SVG

# Small repetition code circuit with noise
circuit = stim.Circuit('''
R 0 1 2
DEPOLARIZE1(0.02) 0 1 2
CNOT 0 1
DEPOLARIZE2(0.02) 0 1
CNOT 2 1
DEPOLARIZE2(0.02) 2 1
M 1
DETECTOR rec[-1]
M 0 2
OBSERVABLE_INCLUDE(0) rec[-2] rec[-1]
''')

# Detector Error Model
dem = circuit.detector_error_model()
matcher = pm.Matching.from_detector_error_model(dem)

# Sample shots
sampler = circuit.compile_detector_sampler()
dets, obs = sampler.sample(10000, separate_observables=True)

# Decode
pred = matcher.decode_batch(dets).reshape(-1, 1)
p_L = np.count_nonzero(pred ^ obs) / len(pred)
print("Logical error rate:", p_L)

# Draw as SVG
svg = circuit.diagram_svg()
SVG(svg)


Logical error rate: 0.0469


AttributeError: 'stim._stim_sse2.Circuit' object has no attribute 'diagram_svg'

In [1]:
# Try to draw SVG; if unavailable, auto-upgrade stim and retry; else fall back to ASCII.
import sys, subprocess, importlib
import stim
from IPython.display import SVG, display

def draw_stim_circuit_svg_or_ascii(circuit: stim.Circuit):
    try:
        return display(SVG(circuit.diagram_svg()))
    except AttributeError:
        # Upgrade stim and retry
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "stim"], stdout=subprocess.DEVNULL)
        import stim as _stim
        importlib.reload(_stim)
        try:
            return display(SVG(circuit.diagram_svg()))
        except Exception:
            print(_stim.__version__)
            print(circuit.diagram("timeline"))

# --- Your circuit ---
circuit = stim.Circuit('''
R 0 1 2
DEPOLARIZE1(0.02) 0 1 2
CNOT 0 1
DEPOLARIZE2(0.02) 0 1
CNOT 2 1
DEPOLARIZE2(0.02) 2 1
M 1
DETECTOR rec[-1]
M 0 2
OBSERVABLE_INCLUDE(0) rec[-2] rec[-1]
''')

draw_stim_circuit_svg_or_ascii(circuit)


1.15.0
<svg viewBox="0 0 992 288"  version="1.1" xmlns="http://www.w3.org/2000/svg">
<g id="qubit_lines">
<path id="qubit_line:0" d="M64,64 L960,64 " stroke="black"/>
<text dominant-baseline="central" text-anchor="end" font-family="monospace" font-size="12" x="64" y="64">q0</text>
<path id="qubit_line:1" d="M64,128 L960,128 " stroke="black"/>
<text dominant-baseline="central" text-anchor="end" font-family="monospace" font-size="12" x="64" y="128">q1</text>
<path id="qubit_line:2" d="M64,192 L960,192 " stroke="black"/>
<text dominant-baseline="central" text-anchor="end" font-family="monospace" font-size="12" x="64" y="192">q2</text>
</g>
<rect x="80" y="48" width="32" height="32" stroke="black" fill="black"/>
<text dominant-baseline="central" text-anchor="middle" font-family="monospace" font-size="30" x="96" y="64" fill="white">R</text>
<rect x="80" y="112" width="32" height="32" stroke="black" fill="black"/>
<text dominant-baseline="central" text-anchor="middle" font-family="monospace"